# Imports

In [1]:
# Packages
import pandas as pd

# Helper functions
from ipynb.fs.full.functions import *

# Other notebooks for combining code
from ipynb.fs.full.semrush_call import execute_call
from ipynb.fs.full.text_division_and_styling import LoopGetHeaderCountFromURL, LoopGetListsCountFromURL

# Executing of functions
## Variables for keyword search

In [2]:
keyword = 'bouwmaterialen' # What keyword do you want to call for?
result_count = '50' # How many keyword results do you want to gather data for?

# Do you want to overwrite the SEMrush keyword call if it has already been executed before?
# This is to save on SEMrush token usage
potential_overwrite = False

# Do you want to save the result of the end dataframe?
save = True

## Creating dataframes

In [3]:
def ExecuteAllCalls():
    # Define variables for the SEMrush call and gather keyword data
    export_columns = 'Dn,Ur'
    endpoint_url = 'https://api.semrush.com/'
    main_dataframe = execute_call(keyword, result_count, export_columns, endpoint_url, potential_overwrite)
    
    # Executing functions from other notebooks to gather data
    print("0% - Executing LoopGetHeaderCountFromURL")
    main_dataframe['HeaderCount'] = LoopGetHeaderCountFromURL(main_dataframe)
    print("50% - Executing LoopGetListsCountFromURL")
    main_dataframe['ListsCount'] = LoopGetListsCountFromURL(main_dataframe)
    
    # Set correct index, potentially save the output and return the dataframe
    main_dataframe = main_dataframe.set_index('Number')
    main_dataframe_hash = hash_variable(main_dataframe)
    if save == True:
        main_dataframe.to_csv(f"calls\dataframe-{main_dataframe_hash}.csv")
    print("100% - Done")
    return main_dataframe
        
main_dataframe = ExecuteAllCalls()
main_dataframe

Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Wed Apr 28 12:38:37 2021
0% - Executing LoopGetHeaderCountFromURL
50% - Executing LoopGetListsCountFromURL
100% - Done


,Dn,Ur,HeaderCount,ListsCount
Number,,,,
0,bouwmaat.nl,https://www.bouwmaat.nl/bouwmaterialen,4.0,31.0
1,hornbach.nl,https://www.hornbach.nl/shop/Bouwmateriaal/S44...,10.0,17.0
2,bouwbestel.nl,https://www.bouwbestel.nl/bouwmaterialen.html,2.0,7.0
3,online-bouwmaterialen.nl,https://www.online-bouwmaterialen.nl/,3.0,31.0
4,bmn.nl,https://www.bmn.nl/,3.0,12.0
5,bouwonline.com,https://www.bouwonline.com/,7.0,7.0
6,bol.com,https://www.bol.com/nl/m/bouwmaterialen/,0.0,0.0
7,pontmeyer.nl,https://www.pontmeyer.nl/,4.0,1.0
8,bouwmaterialenkopen.com,https://www.bouwmaterialenkopen.com/,8.0,3.0
